In [30]:
from bs4 import BeautifulSoup

html = requests.get("https://www.yelp.com/biz/my-burma-davis?osq=Restaurants")
soup = BeautifulSoup(html.text)

name = soup.find('h1').text
print(name)

My Burma


In [31]:
import re

rating_tag = soup.find('div', attrs={'aria-label': re.compile('star rating')})
rating = rating_tag['aria-label']
print(rating)

4.5 star rating


In [32]:
review_count = soup.find('span', string=re.compile('reviews')).text
print(review_count)

(480 reviews)


In [33]:
website_sibling = soup.find('p', string="Business website")
if website_sibling:
    website = website_sibling.next_sibling.text
    print(website)

my-burma.com


In [34]:
phone_no_sibling = soup.find('p', string="Phone number")
if phone_no_sibling:
    phone_no = phone_no_sibling.next_sibling.text
    print(phone_no)

(530) 231-5229


In [35]:
address_sibling = soup.find('a', string="Get Directions")
if address_sibling:
    address = address_sibling.parent.next_sibling.text
    print(address)

500 1st St Ste 11 Davis, CA 95616


In [45]:
import re
from bs4 import BeautifulSoup
import requests

html = requests.get("https://www.yelp.com/biz/my-burma-davis?osq=Restaurants")
soup = BeautifulSoup(html.text, 'html.parser')

name = soup.find('h1').text
print(name)

website_sibling = soup.find('p', string="Business website")
if website_sibling:
    website = website_sibling.next_sibling.text
    print(website)

phone_no_sibling = soup.find('p', string="Phone number")
if phone_no_sibling:
    phone_no = phone_no_sibling.next_sibling.text
    print(phone_no)

address_sibling = soup.find('a', string="Get Directions")
if address_sibling:
    address = address_sibling.parent.next_sibling.text
    print(address)

rating_tag = soup.find('div', attrs={'aria-label': re.compile('star rating')})
rating = rating_tag['aria-label']
print(rating)

review_count = soup.find('span', string=re.compile('reviews')).text
print(review_count)

My Burma
my-burma.com
(530) 231-5229
500 1st St Ste 11 Davis, CA 95616
4.5 star rating
(480 reviews)


In [57]:
search_url = "https://www.yelp.com/search/snippet?find_desc=Restaurants&find_loc=San+Francisco%2C+CA%2C+United+States&request_origin=user"
search_response = requests.get(search_url)
search_results = search_response.json()['searchPageProps']['mainContentComponentsListProps']

for result in search_results:
    if result['searchResultLayoutType'] == "iaResult":
        print(result['searchResultBusiness']['name'])
        print(result['searchResultBusiness']['reviewCount'])
        print(result['searchResultBusiness']['rating'])
        print("https://www.yelp.com" + result['searchResultBusiness']['businessUrl'])
        print("--------")

Memento SF
29
4.9
https://www.yelp.com/biz/memento-sf-san-francisco-3?osq=Restaurants
--------
NONO
76
4.4
https://www.yelp.com/biz/nono-san-francisco?osq=Restaurants
--------
Marufuku Ramen
4921
4.5
https://www.yelp.com/biz/marufuku-ramen-san-francisco-5?osq=Restaurants
--------
Kothai Republic
235
4.7
https://www.yelp.com/biz/kothai-republic-san-francisco?osq=Restaurants
--------
Yakitori Edomasa
41
4.2
https://www.yelp.com/biz/yakitori-edomasa-san-francisco?osq=Restaurants
--------
Bottega
957
4.3
https://www.yelp.com/biz/bottega-san-francisco-2?osq=Restaurants
--------
The Snug
426
4.0
https://www.yelp.com/biz/the-snug-san-francisco?osq=Restaurants
--------
Otra
163
4.5
https://www.yelp.com/biz/otra-san-francisco-3?osq=Restaurants
--------
C.Q. Noodles
145
4.5
https://www.yelp.com/biz/c-q-noodles-san-francisco-2?osq=Restaurants
--------
Katsuo + Kombu
117
4.4
https://www.yelp.com/biz/katsuo-kombu-san-francisco?osq=Restaurants
--------


In [1]:
import requests

url = "https://api.yelp.com/v3/businesses/search?sort_by=best_match&limit=20"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"error": {"code": "VALIDATION_ERROR", "description": "Authorization is a required parameter.", "field": "Authorization", "instance": null}}


In [2]:
import requests
from bs4 import BeautifulSoup
import re

# Function to get business information from a page
def get_business_info(page_url):
    response = requests.get(page_url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    business_name_elements = soup.find_all('h3', class_='css-1agk4wl')

    business_info_list = []
    for business_name_element in business_name_elements:
        business_name = business_name_element.text.strip()
        business_link = business_name_element.find('a')['href']

        # Make a request for the individual business page
        business_page_url = f"https://www.yelp.com{business_link}"
        business_page_response = requests.get(business_page_url)
        business_page_soup = BeautifulSoup(business_page_response.text, 'html.parser')

        # Extract the rating from the individual business page
        rating_div = business_page_soup.find('div', {'aria-label': re.compile(' star rating')})
        rating = rating_div['aria-label'] if rating_div else None

        business_info_list.append({'name': business_name, 'link': business_link, 'rating': rating})

    return business_info_list

# URL template with the start parameter
url_templateSF = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=San+Francisco%2C+CA&start={}"

# Number of pages to scrape
num_pages = 4  # Change this to the desired number of pages

# List to store all business information
all_business_infoSF = []

# Iterate through pages
for page_number in range(0, num_pages * 10, 10):
    page_url = url_templateSF.format(page_number)
    business_info = get_business_info(page_url)
    all_business_infoSF.extend(business_info)

# Print all business names, links, and ratings
for business in all_business_infoSF:
    print(f"Business Name: {business['name']}")
    print(f"   Business Link: {business['link']}")
    print(f"   Rating: {business['rating']}\n")


In [5]:
import requests
from bs4 import BeautifulSoup
import time

# Function to get reviews from a business page
def get_reviews(business_page_url):
    reviews_list = []

    # Initial request for the business page
    response = requests.get(business_page_url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract reviews from the initial page
    reviews_divs = soup.find_all('div', class_='css-9ul5p9')
    reviews = [div.find('span', class_='raw__09f24__T4Ezm').text.strip() if div.find('span', class_='raw__09f24__T4Ezm') else None for div in reviews_divs]
    reviews_list.extend([review for review in reviews if review])

    # Check for additional pages and retrieve reviews
    pagination_div = soup.find('div', class_='pagination__09f24__2M9a_')
    if pagination_div:
        next_page_link = pagination_div.find('a', class_='pagination-link_component__09f24__gSSRj', text='Next')['href']
        while next_page_link:
            next_page_url = f"https://www.yelp.com{next_page_link}"
            response = requests.get(next_page_url)
            html_content = response.text
            soup = BeautifulSoup(html_content, 'html.parser')

            # Extract reviews from the current page
            reviews_divs = soup.find_all('div', class_='css-9ul5p9')
            reviews = [div.find('span', class_='raw__09f24__T4Ezm').text.strip() if div.find('span', class_='raw__09f24__T4Ezm') else None for div in reviews_divs]
            reviews_list.extend([review for review in reviews if review])

            # Check for the next page
            next_page_link = pagination_div.find('a', class_='pagination-link_component__09f24__gSSRj', text='Next')['href'] if pagination_div.find('a', class_='pagination-link_component__09f24__gSSRj', text='Next') else None

            # Add a delay to avoid being blocked
            time.sleep(1)

    return reviews_list

# URL template with the start parameter for reviews
url_template_reviewsSF = "https://www.yelp.com{}&start={}"

# Number of pages to scrape for reviews
num_pages_reviews = 4  # Change this to the desired number of pages

all_reviewsSF = []

# Iterate through business information for reviews
for business in all_business_infoSF:
    business_linkSF = business['link']

    # List to store all reviews for the current business
    business_reviewsSF = []

    # Iterate through pages for reviews
    for page_number in range(0, num_pages_reviews * 10, 10):
        page_url = url_template_reviewsSF.format(business_linkSF, page_number)
        reviewsSF = get_reviews(page_url)

        # Add reviews to the list for the current business
        business_reviewsSF.extend(reviewsSF)

        # Add a delay to avoid being blocked
        time.sleep(1)

    # Print reviews for the current business
    print(f"Business Link: {business_linkSF}")
    print(f"Reviews: {business_reviewsSF}\n")

    # Add reviews for the current business to the main list
    all_reviewsSF.extend(business_reviewsSF)


Business Link: /biz/bottega-san-francisco-2?osq=Restaurants
Reviews: ['Went to Bottega for a date night and let me tell you it was amazing. Had a reservation so did not have to wait long, once seated the waiter attended to us right away and we were able to order. Food is amazing and so fresh! Drinks were good too. Highly recommend for a cute little date night.', "Everything was great! Food, menu options, staff & vibe. It's nice to have a traditional Italian restaurant that isn't stuffy or way over priced", 'We had an amazing experience at Bottega! The food was delicious and the service was exceptional! We tried several dishes and none of them disappointed! I would highly recommend and I will definitely be returning!', "I recently had the pleasure of dining at Bottega Restraint and I must say, it was an extraordinary experience.As for the food, every dish was a work of art. The presentation was impeccable, and each bite was bursting with flavor. From the appetizers to the desserts, ever

In [21]:
import requests
from bs4 import BeautifulSoup
import re

# Function to get business information from a page
def get_business_info(page_url):
    response = requests.get(page_url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    business_name_elements = soup.find_all('h3', class_='css-1agk4wl')

    business_info_list = []
    for business_name_element in business_name_elements:
        business_name = business_name_element.text.strip()
        business_link = business_name_element.find('a')['href']

        # Make a request for the individual business page
        business_page_url = f"https://www.yelp.com{business_link}"
        business_page_response = requests.get(business_page_url)
        business_page_soup = BeautifulSoup(business_page_response.text, 'html.parser')

        # Extract the rating from the individual business page
        rating_div = business_page_soup.find('div', {'aria-label': re.compile(' star rating')})
        rating = rating_div['aria-label'] if rating_div else None

        business_info_list.append({'name': business_name, 'link': business_link, 'rating': rating})

    return business_info_list

# URL template with the start parameter
url_templateLA = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&start={}"

# Number of pages to scrape
num_pages = 4  # Change this to the desired number of pages

# List to store all business information
all_business_infoLA = []

# Iterate through pages
for page_number in range(0, num_pages * 10, 10):
    page_url = url_templateLA.format(page_number)
    business_info = get_business_info(page_url)
    all_business_infoLA.extend(business_info)

# Print all business names, links, and ratings
for business in all_business_infoLA:
    print(f"Business Name: {business['name']}")
    print(f"   Business Link: {business['link']}")
    print(f"   Rating: {business['rating']}\n")

In [11]:
import requests
from bs4 import BeautifulSoup
import time

# Function to get reviews from a business page
def get_reviews(business_page_url):
    reviews_list = []

    # Initial request for the business page
    response = requests.get(business_page_url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract reviews from the initial page
    reviews_divs = soup.find_all('div', class_='css-9ul5p9')
    reviews = [div.find('span', class_='raw__09f24__T4Ezm').text.strip() if div.find('span', class_='raw__09f24__T4Ezm') else None for div in reviews_divs]
    reviews_list.extend([review for review in reviews if review])

    # Check for additional pages and retrieve reviews
    pagination_div = soup.find('div', class_='pagination__09f24__2M9a_')
    if pagination_div:
        next_page_link = pagination_div.find('a', class_='pagination-link_component__09f24__gSSRj', text='Next')['href']
        while next_page_link:
            next_page_url = f"https://www.yelp.com{next_page_link}"
            response = requests.get(next_page_url)
            html_content = response.text
            soup = BeautifulSoup(html_content, 'html.parser')

            # Extract reviews from the current page
            reviews_divs = soup.find_all('div', class_='css-9ul5p9')
            reviews = [div.find('span', class_='raw__09f24__T4Ezm').text.strip() if div.find('span', class_='raw__09f24__T4Ezm') else None for div in reviews_divs]
            reviews_list.extend([review for review in reviews if review])

            # Check for the next page
            next_page_link = pagination_div.find('a', class_='pagination-link_component__09f24__gSSRj', text='Next')['href'] if pagination_div.find('a', class_='pagination-link_component__09f24__gSSRj', text='Next') else None

            # Add a delay to avoid being blocked
            time.sleep(1)

    return reviews_list

# URL template with the start parameter for reviews
url_template_reviewsLA = "https://www.yelp.com{}&start={}"

# Number of pages to scrape for reviews
num_pages_reviews = 4  # Change this to the desired number of pages

all_reviewsLA = []

# Iterate through business information for reviews
for business in all_business_infoLA:
    business_linkLA = business['link']

    # List to store all reviews for the current business
    business_reviewsLA = []

    # Iterate through pages for reviews
    for page_number in range(0, num_pages_reviews * 10, 10):
        page_url = url_template_reviewsLA.format(business_linkLA, page_number)
        reviewsLA = get_reviews(page_url)

        # Add reviews to the list for the current business
        business_reviewsLA.extend(reviewsLA)

        # Add a delay to avoid being blocked
        time.sleep(1)

    # Print reviews for the current business
    print(f"Business Link: {business_linkLA}")
    print(f"Reviews: {business_reviewsLA}\n")

    # Add reviews for the current business to the main list
    all_reviewsLA.extend(business_reviewsLA)

In [12]:
# Function to count words in a given text
def count_words(text):
    words = text.split()  # Split the text into words
    return len(words)

# Calculate total words in the entire review list
total_wordsSF = sum(count_words(review) for review in all_reviewsSF)
total_wordsLA = sum(count_words(review) for review in all_reviewsLA)

print(f"Total words in all reviews: {total_wordsSF}")
print(f"Total words in all reviews: {total_wordsLA}")

Total words in all reviews: 65273
Total words in all reviews: 0


In [2]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Combine all reviews into a single string
all_textSF = ' '.join(all_reviewsSF)

# Generate a word cloud
wordcloud = WordCloud(width=800, height=400, max_words=200, background_color='white').generate(all_textSF)

# Display the word cloud using Matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

NameError: name 'all_reviewsSF' is not defined